# Biblioteca

In [ ]:
!pip install psycopg2-binary
!pip install sqlalchemy sqlalchemy-schemadisplay pydot
!pip install pydot
import psycopg2
import pandas as pd
from pathlib import Path



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Utilidades

In [59]:
def get_dataframe_from_csv(file_path, delimiter=','):
    try:
        df = pd.read_csv(file_path, dtype=str, delimiter=delimiter)
        print(f"Arquivo CSV '{file_path}' lido com sucesso.")
        return df
    except Exception as e:
        print(f"Erro ao ler o arquivo CSV: {e}")
        return pd.DataFrame()  # Retorna um DataFrame vazio em caso de erro

# Conexão

In [ ]:
db_config = {
    "host": "localhost",
    "port": "5432",
    "database": "tebd_db",
    "user": "tebd",
    "password": "tebd"
}

conn = None
cur = None

# --- Conectar ao Banco de Dados ---
try:
    conn = psycopg2.connect(**db_config)
    print("Conexão com o PostgreSQL bem-sucedida!")
    cur = conn.cursor()
except psycopg2.Error as e:
    print(f"Erro ao conectar ao PostgreSQL: {e}")


Conexão com o PostgreSQL bem-sucedida!


In [14]:
# --- Definição dos comandos SQL ---
sql_create_municipio = """
    CREATE TABLE IF NOT EXISTS Municipio (
        municipio_id VARCHAR(6) PRIMARY KEY UNIQUE,
        municipio_nome VARCHAR(100)
    );
"""

sql_create_estabelecimento = """
    CREATE TABLE IF NOT EXISTS Estabelecimento (
        cnes_id VARCHAR(7) PRIMARY KEY UNIQUE,
        cnpj VARCHAR(15),
        municipio_id VARCHAR(6) REFERENCES Municipio(municipio_id)
    );
"""

sql_create_procedimento = """
    CREATE TABLE IF NOT EXISTS Procedimento (
        procedimento_id VARCHAR(10) PRIMARY KEY UNIQUE,
        procedimento_nome VARCHAR(255)
    );
"""

sql_create_cid = """
    CREATE TABLE IF NOT EXISTS Cid (
        cid_id VARCHAR(4) PRIMARY KEY UNIQUE,
        cid_nome VARCHAR(255)
    );
"""

sql_create_pessoa = """
    CREATE TABLE IF NOT EXISTS Pessoa (
        pessoa_id SERIAL PRIMARY KEY UNIQUE,
        nasc DATE,
        sexo VARCHAR(1),
        idade NUMERIC(2),
        raca_cor VARCHAR(4),
        etnia VARCHAR(4),
        cod_idade VARCHAR(1)
    );
"""

sql_create_internamento = """
    CREATE TABLE IF NOT EXISTS Internamento (
        internamento_id VARCHAR(13) PRIMARY KEY UNIQUE,
        qt_diarias NUMERIC(3),
        val_sh NUMERIC(13, 2),
        val_sp NUMERIC(13, 2),
        val_tot NUMERIC(14, 2),
        dt_inter DATE,
        morte NUMERIC(1),
        cid_morte VARCHAR(4),
        cnes_id VARCHAR(7) REFERENCES Estabelecimento(cnes_id),
        procedimento_id VARCHAR(10) REFERENCES Procedimento(procedimento_id),
        cid_id VARCHAR(4) REFERENCES Cid(cid_id),
        pessoa_id INT REFERENCES Pessoa(pessoa_id)
    );
"""

if conn is not None and cur is not None:
    try:
        cur.execute(sql_create_municipio)
        print("Tabela 'Municipio' criada com sucesso.")

        cur.execute(sql_create_estabelecimento)
        print("Tabela 'Estabelecimento' criada com sucesso.")

        cur.execute(sql_create_procedimento)
        print("Tabela 'Procedimento' criada com sucesso.")

        cur.execute(sql_create_cid)
        print("Tabela 'Cid' criada com sucesso.")

        cur.execute(sql_create_pessoa)
        print("Tabela 'Pessoa' criada com sucesso.")

        cur.execute(sql_create_internamento)
        print("Tabela 'Internamento' criada com sucesso.")

        conn.commit()
    except psycopg2.Error as e:
        print(f"Erro ao criar as tabelas: {e}")
        conn.rollback()

Tabela 'Municipio' criada com sucesso.
Tabela 'Estabelecimento' criada com sucesso.
Tabela 'Procedimento' criada com sucesso.
Tabela 'Cid' criada com sucesso.
Tabela 'Pessoa' criada com sucesso.
Tabela 'Internamento' criada com sucesso.


# Inserção nas tabelas

## Municipios

In [32]:
df_municipios = get_dataframe_from_csv('data/municipios_tratado.csv')

Arquivo CSV 'data/municipios_tratado.csv' lido com sucesso.


## Estabelecimentos

In [55]:
df_estabelecimentos = get_dataframe_from_csv('data/CNES_tratado.csv')

Arquivo CSV 'data/CNES_tratado.csv' lido com sucesso.


## Procedimentos

In [60]:
df_procedimentos = get_dataframe_from_csv('data/procedimentos_tratado.csv', delimiter=';')

Arquivo CSV 'data/procedimentos_tratado.csv' lido com sucesso.


## Cid

In [62]:
df_cid = get_dataframe_from_csv('data/cid_tratado.csv', delimiter=';')

Arquivo CSV 'data/cid_tratado.csv' lido com sucesso.


## Pessoa e Internamento

O dataframe para pessoa e internamento é o mesmo.

In [69]:
DEBUG = True

anos = [2019, 2020, 2021, 2022, 2023, 2024]

df_internamentos_list = []

if DEBUG:
    df_internamento = get_dataframe_from_csv('data/arquivo_500linhas.csv')
    df_internamentos_list.append(df_internamento)
else:
    for ano in anos:
        df_internamento = get_dataframe_from_csv(f'data/{ano}.csv')
        df_internamentos_list.append(df_internamento)


Arquivo CSV 'data/arquivo_500linhas.csv' lido com sucesso.


### Pessoa

### Internamento

In [70]:
sql_insert_municipio = "INSERT INTO Municipio (municipio_id) VALUES (%s) ON CONFLICT (municipio_id) DO NOTHING;"
sql_insert_estabelecimento = "INSERT INTO Estabelecimento (cnes_id) VALUES (%s) ON CONFLICT (cnes_id) DO NOTHING;"
sql_insert_procedimento = "INSERT INTO Procedimento (procedimento_id) VALUES (%s) ON CONFLICT (procedimento_id) DO NOTHING;"
sql_insert_cid = "INSERT INTO Cid (cid_id) VALUES (%s) ON CONFLICT (cid_id) DO NOTHING;"
sql_insert_pessoa = "INSERT INTO Pessoa (nasc, sexo, idade, raca_cor, etnia) VALUES (%s, %s, %s, %s, %s) RETURNING pessoa_id;"
sql_insert_internamento = """
    INSERT INTO Internamento (
        internamento_id, qt_diarias, val_sh, val_sp, val_tot, dt_inter, morte, cid_morte,
        cnes_id, procedimento_id, cid_id, pessoa_id
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
try:
    if conn:
        try:
            df = pd.read_csv('arquivo_500linhas.csv')

            for cnes in df['CNES'].unique():
                cur.execute(sql_insert_estabelecimento, (int(cnes),))

            for proc in df['PROC_REA'].unique():
                cur.execute(sql_insert_procedimento, (str(proc),))

            for cid in df['DIAG_PRINC'].unique():
                if pd.notna(cid) and cid != '0':
                    cur.execute(sql_insert_cid, (str(cid),))

            conn.commit()

            for index, row in df.iterrows():
                cur.execute(
                    sql_insert_pessoa,
                    (int(row['NASC']), int(row['SEXO']), int(row['IDADE']), str(row['RACA_COR']), str(row['ETNIA']))
                )
                pessoa_id = cur.fetchone()[0]

                cur.execute(
                    sql_insert_internamento,
                    (
                        int(row['N_AIH']), int(row['QT_DIARIAS']), row['VAL_SH'], row['VAL_SP'], row['VAL_TOT'],
                        int(row['DT_INTER']), int(row['MORTE']), str(row['CID_MORTE']), int(row['CNES']),
                        str(row['PROC_REA']), str(row['DIAG_PRINC']), pessoa_id
                    )
                )

            conn.commit()

        except psycopg2.Error as e:
            print(f"Erro ao inserir dados: {e}")
            conn.rollback()

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()
        print("Conexão com o PostgreSQL encerrada.")


Conexão com o PostgreSQL encerrada.


FileNotFoundError: [Errno 2] No such file or directory: 'arquivo_500linhas.csv'